In [1]:
import gdown
import pandas as pd

# ID của file Google Drive
file_id = "1jkuOxJ2zl-3bbXPE4UXELKGoisgKC8bx"
url = f"https://drive.google.com/uc?id={file_id}"

# Tải file tạm thời trên Google Colab
output_path = "/content/processed_arxiv_cs_papers_2.csv"
gdown.download(url, output_path, quiet=False)

# Đọc file CSV
raw_df = pd.read_csv(output_path)

# Hiển thị dữ liệu
print(raw_df.head())

Downloading...
From (original): https://drive.google.com/uc?id=1jkuOxJ2zl-3bbXPE4UXELKGoisgKC8bx
From (redirected): https://drive.google.com/uc?id=1jkuOxJ2zl-3bbXPE4UXELKGoisgKC8bx&confirm=t&uuid=0046a3f1-225a-4fcb-94a3-b3e209c39d70
To: /content/processed_arxiv_cs_papers_2.csv
100%|██████████| 311M/311M [00:02<00:00, 131MB/s]


                                                text  cs.AI  cs.CL  cs.CR  \
0  gaussianworld gaussian world model streaming o...      1      0      0   
1  gaussianad gaussiancentric endtoend autonomous...      1      0      0   
2  inverse reinforcement learning estimating expe...      1      0      0   
3  apollo exploration video understanding large m...      1      0      0   
4  library learning neural operator present neura...      1      0      0   

   cs.CV  cs.CY  cs.DC  cs.DM  cs.DS  cs.HC  ...  cs.IT  cs.LG  cs.LO  cs.NA  \
0      1      0      0      0      0      0  ...      0      1      0      0   
1      1      0      0      0      0      0  ...      0      1      0      0   
2      0      0      0      0      0      0  ...      0      1      0      0   
3      1      0      0      0      0      0  ...      0      0      0      0   
4      0      0      0      0      0      0  ...      0      1      0      0   

   cs.NE  cs.NI  cs.RO  cs.SE  cs.SI  cs.SY  
0      0  

In [2]:
pip install transformers tensorflow pandas scikit-learn

### Cài đặt các thư viện cần thiết

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf

### Đọc dữ liệu

In [4]:
# file_path = '/content/drive/My Drive/Data Science - Cuối kì/processed_arxiv_cs_papers_2.csv'
# raw_df = pd.read_csv(file_path)

### Phân chia dữ liệu cho việc train và test

In [5]:
text = raw_df["text"]
labels = np.asarray(raw_df[raw_df.columns[1:]])
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42)

### Tokenize dữ liệu huấn luyện và kiểm tra

In [6]:
# Khởi tạo tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

# Định nghĩa hàm tokenization
def tokenize_data(texts):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=100, return_tensors='tf')

train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

### Tải và cấu hình mô hình SciBERT

In [7]:
model = TFBertForSequenceClassification.from_pretrained(
    'allenai/scibert_scivocab_uncased',          # tên mô hình
    num_labels=y_train.shape[1],                 # số lượng nhãn
    problem_type="multi_label_classification",   # loại bài toán
    from_pt=True                                 # sử dụng trọng số từ mô hình PyTorch và chuyển đổi sang TensorFlow.
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),     # tối ưu hóa Adam
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # hàm mất mát
    metrics=['accuracy']                                        # sử dụng accuracy để theo dõi độ chính xác trong quá trình huấn luyện.
)

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Huấn luyện mô hình

In [8]:
history = model.fit(
    train_encodings['input_ids'],   # đầu vào của mô hình là các ID token đã được mã hóa từ văn bản ban đầu
    y_train,                        # nhãn của dữ liệu
    validation_split=0.1,           # nhãn huấn luyện của dữ liệu, dưới dạng ma trận nhị phân
    epochs=2,                       # số lượng epoch
    batch_size=8                    # kích thước batch
)

Epoch 1/2
28265/28265 [==============================] - 5692s 200ms/step - loss: 0.1118 - accuracy: 0.5848 - val_loss: 0.1039 - val_accuracy: 0.5849
Epoch 2/2
28265/28265 [==============================] - 5651s 200ms/step - loss: 0.0929 - accuracy: 0.6277 - val_loss: 0.1019 - val_accuracy: 0.5995


### Đánh giá mô hình

In [9]:
results = model.evaluate(test_encodings['input_ids'], y_test, batch_size=8)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

7852/7852 [==============================] - 549s 70ms/step - loss: 0.1009 - accuracy: 0.6041
Test Loss: 0.10092116892337799, Test Accuracy: 0.6041235327720642


In [10]:
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(labels)
# # Dự đoán nhãn cho tập kiểm tra
# y_pred = model.predict(test_encodings['input_ids']).logits
# y_pred = tf.nn.sigmoid(y_pred)            # Chuyển logits thành xác suất
# y_pred = tf.cast(y_pred > 0.5, tf.int32)  # Chuyển xác suất thành nhãn nhị phân

# # Giải mã nhãn dự đoán
# predicted_labels = mlb.inverse_transform(y_pred.numpy())
# print(predicted_labels[:5])  # Xem 5 kết quả đầu tiên

In [11]:
# from sklearn.metrics import classification_report, hamming_loss, accuracy_score

# # Đánh giá mô hình
# print(f"Accuracy: {accuracy_score(y_test, y_pred.numpy())}")
# print(f"Hamming Loss: {hamming_loss(y_test, y_pred.numpy())}")
# print(classification_report(y_test, y_pred.numpy(), target_names=mlb.classes_))

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Lưu mô hình vào Google Drive
model_save_path = '/content/drive/MyDrive/model_NMKHDL/'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/drive/MyDrive/model_NMKHDL/
